In [1]:
#!/usr/bin/env pyhton3
# -*- coding 

from sklearn import datasets
from sklearn.model_selection import train_test_split
###-----------------
### Import Libraries
###-----------------
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from utils.helper import fn_plot_tf_hist, fn_plot_confusion_matrix
import gc

from sklearn.metrics import mean_squared_error,accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

import warnings
warnings.filterwarnings('ignore')
from utils.helper import fn_plot_tf_hist, fn_plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
# Global variables
inpDir = './input' # Input Stored here
outDir = './ouput' # output Here
modelDir = './models'# to save Models
subDir = 'text_gen' # sub dir by dataset
RANDOM_STATE = 24
np.random.RandomState(seed = RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
#rng = np.random.default_rng(seed = RANDOM_STATE)
#N_SAMPLE = 1000
TEST_SIZE = 0.2 # 18147 - 56*256
ALPHA = 0.001 # learning rate
NOISE = 0.2 # Error
EPOCHS = 30
BATCH_SIZE = 64 # default for tensorflow

PATIENCE = 10
LR_FACTOR = 0.1
LR_PATIENCE = 10

In [3]:
#set the plotting parameters
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'
         }
    #'text.usetex':True,

plt.rcParams.update(params)
CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid')


In [4]:
fileName = 'shakespeare.txt'
filePath = os.path.join(inpDir, subDir, fileName)

filePath

'./input\\text_gen\\shakespeare.txt'

In [5]:
text = open(filePath, 'rb').read().decode(encoding='utf-8')
len(text)

1115395

In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


# character based modelling

In [8]:
vocab = sorted(set(text))
len(vocab)

65

In [9]:
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab) #in array i can refer element by idx

In [10]:
text_as_int = np.array([char2idx [c] for c in text])
text_as_int.shape

(1115395,)

In [11]:
type(text_as_int)

numpy.ndarray

In [12]:
seq_length = 100
example_per_epoch = len(text) // (seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(i.numpy(), end=':')
    
    print(idx2char[i.numpy()])
   

18:F
47:i
56:r
57:s
58:t
1: 
15:C
47:i
58:t
47:i


In [13]:
sequences = char_dataset.batch(seq_length +1, drop_remainder = True)
for item in sequences.take(2):
    print(item)
    print(repr(''.join(idx2char[item.numpy()])))

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1] # First hundred characters
    target_text = chunk[1:] # offset one as target
    return input_text, target_text
dataset = sequences.map(split_input_target)


In [15]:
for inp_ex, tar_ex in dataset.take(2):
    print(repr(''.join(idx2char[inp_ex.numpy()])))
    print(repr(''.join(idx2char[tar_ex.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [16]:
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,
                                            drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [17]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [35]:
def build_model(vocab_size, embedding_dim, 
                rnn_units, batch_size = BATCH_SIZE):
    return tf.keras.Sequential([
        tf.keras.layers.Input(shape = (None, ),
                              batch_size = batch_size),
        tf.keras.layers.Embedding(vocab_size,
                                  embedding_dim),
        tf.keras.layers.GRU(rnn_units, 
                            return_sequences=True, 
                           stateful=True,
                           recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
                               ])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 100)                 │       1,010,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,010,201 (3.85 MB)

 Trainable params: 1,010,201 (3.85 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model = build_model(vocab_size, embedding_dim, rnn_units,
               BATCH_SIZE)

In [20]:
for inp_ex, tar_ex in dataset.take(1):
    ex_pred = model(inp_ex)

In [21]:
ex_pred.shape

TensorShape([64, 100, 65])

In [25]:
# Why we sample

In [31]:
sample_indices = tf.random.categorical(ex_pred[0], num_samples = 1)
print(sample_indices.shape)

(100, 1)


In [33]:
sample_indices = tf.squeeze(sample_indices, axis= -1).numpy()
sample_indices

array([14,  1, 27, 46, 28, 51,  2, 50, 17, 30, 27, 63, 41, 48, 42, 64, 63,
       15, 32, 10,  9, 21, 56, 39, 43, 37, 19, 21, 31, 19, 17,  8,  1, 29,
       25, 10,  3, 59, 15, 62, 24, 26, 37, 38, 56, 37, 63,  5,  0,  0, 35,
        4, 52, 44, 28, 41, 12, 18, 17, 12, 46,  6, 46,  7, 23, 19, 22, 18,
       37, 59,  3, 42, 45, 46, 51, 51, 48, 16, 27, 19,  5,  3, 16, 32, 21,
       22, 16, 31, 24, 50, 27, 39, 45, 21, 33, 59, 11,  9, 44, 40],
      dtype=int64)

In [22]:
h_units = 100 # hidden layers No. of neurons
input_shape = (1,10000) # inputs 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=input_shape))
model.add(tf.keras.layers.SimpleRNN(units = h_units,
                                   activation='tanh'
                                   ))
model.add(tf.keras.layers.Dense(1,
                                   activation='linear',
                                   
                                   ))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 100)                 │       1,010,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,010,201 (3.85 MB)

 Trainable params: 1,010,201 (3.85 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
for ax in model.get_weights():
    print(ax.shape)
print('\n')
print(f'Wax = {model.get_weights()[0].shape}')
print(f'Waa = {model.get_weights()[1].shape}')
print(f'ba = {model.get_weights()[2].shape}')
print(f'Way = {model.get_weights()[3].shape}')
print(f'by = {model.get_weights()[4].shape}')

(10000, 100)
(100, 100)
(100,)
(100, 1)
(1,)


Wax = (10000, 100)
Waa = (100, 100)
ba = (100,)
Way = (100, 1)
by = (1,)


In [24]:
temp_df

NameError: name 'temp_df' is not defined

In [ ]:
time_step = 24
temp_df.shape[0]/time_step


y_idx = np.arange(time_step, temp_df.shape[0], time_step)
y_idx

In [ ]:
y_df = temp_df.iloc[y_idx]
y_df.head()

In [ ]:
y_df.shape

In [ ]:
temp_df = temp_df.iloc[range(len(y_df) * time_step)]
temp_df.shape

In [ ]:
# Reshape to match number of e=rows in y_df

X_temp = np.reshape(temp_df['temp'].values, (y_df.shape[0], time_step))
X_hum = np.reshape(temp_df['hum'].values, (y_df.shape[0], time_step))
X_wind = np.reshape(temp_df['wind'].values, (y_df.shape[0], time_step))
X_vis = np.reshape(temp_df['vis'].values, (y_df.shape[0], time_step))

X_temp.shape, X_hum.shape, X_wind.shape, X_vis.shape

In [ ]:
# remove last column
X_temp  = X_temp[:, :23]
X_hum = X_hum[:,:23]
X_wind = X_wind[:,:23]
X_vis = X_vis[:,:23]

In [ ]:
#X[0], y_df.head(1)

[samples, timesteps, features]

In [ ]:
X_data = np.stack((
    X_temp,
    X_hum,X_vis,
    X_wind))
X_data = np.reshape(X_data, (X_temp.shape[0], X_temp.shape[1], 4))
X_data.shape

In [ ]:
split = int(y_df.shape[0] * (1 - TEST_SIZE))
split

In [ ]:
X_train = X_data[:split]
X_test = X_data[split:]
y_train = y_df['temp'].values[:split]
y_test = y_df['temp'].values[split:]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
h_units = 256 # hidden layers No. of neurons
input_shape = (time_step - 1,X_train.shape[2]) # inputs  (time_Step, feature )
initializer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=input_shape))
model.add(tf.keras.layers.SimpleRNN(units = h_units,
                                    kernel_initializer= initializer,
                                       activation='tanh'
                                   ))
model.add(tf.keras.layers.Dense(1,
                                   activation='linear',
                                   
                                   ))

model.summary()

In [ ]:
model.compile(optimizer = 'adam', # optimisation, loss calculation 
             loss = 'mean_squared_error',
              metrics = [tf.keras.metrics.RootMeanSquaredError(name='RMSE', dtype=None)]
             )
history = model.fit(x = X_train, y = y_train, 
                    validation_data = [X_test, y_test],
                    batch_size = BATCH_SIZE,
                    verbose = 1,
                    epochs =EPOCHS
                   )

In [ ]:
loss_df = pd.DataFrame(history.history)

In [ ]:
loss_df

In [ ]:
fn_plot_tf_hist(loss_df)

In [ ]:
y_tr_pred = model.predict(X_train)
y_ts_pred = model.predict(X_test)

y_pred = np.append(y_tr_pred, y_ts_pred)
res_df = y_df.copy()
res_df['pred'] = y_pred
res_df.head()

In [ ]:
res_df.plot(x = 'datetime', y=['temp', 'pred'])

In [ ]:
chkptPATH = os.path.join(modelDir, subDir, 'all_in_11.keras')
chkpt_cb = tf.keras.callbacks.ModelCheckpoint(chkptPATH,
                                             monitor = 'val_loss',
                                             verbose =2,
                                             save_best_only=True
                                        )

es_cb = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                        verbose =2,
                                        patience= PATIENCE,
                                         restore_best_weights= True
                                        )

lr_cb = tf.keras.callbacks.ReduceLROnPlateau(
                                        monitor = 'val_loss',
                                        factor= LR_FACTOR,
                                        verbose =2,
                                        patience= LR_PATIENCE
                                         )

In [ ]:
history = model.fit(test_ds,#x = X_train, y = y_train, 
                    validation_data = valid_ds,#[X_test, y_test],
                    #batch_size = BATCH_SIZE,
                    verbose = 2,
                    epochs =EPOCHS,
                     callbacks = [chkpt_cb,es_cb,lr_cb]
                   )

In [ ]:
fn_plot_tf_hist(pd.DataFrame(history.history))